In [ ]:
%run glue.py
from fbprophet import Prophet
import numpy as np

In [ ]:
df_popsike.head()

In [ ]:
curr_release = int(df_popsike['release_id'].sample())
df_popsike[df_popsike['release_id'] == curr_release]

In [ ]:
def log_transform(df):
    df['y'] = df['y'] + 1
    df['y'] = np.log(df['y'])
    return df

def inverse_log_transform(df):
    df['yhat'] = np.exp(df['yhat'])-1
    df['yhat_lower'] = np.exp(df['yhat_lower'])-1
    df['yhat_upper'] = np.exp(df['yhat_upper'])-1
    return df

In [ ]:
curr_release_min = pd.DataFrame({
    'ds': df_popsike[df_popsike['release_id'] == curr_release]['year'],
    'y': df_popsike[df_popsike['release_id'] == curr_release]['min']
})
curr_release_min = log_transform(curr_release_min)

curr_release_avg = pd.DataFrame({
    'ds': df_popsike[df_popsike['release_id'] == curr_release]['year'],
    'y': df_popsike[df_popsike['release_id'] == curr_release]['avg']
})
curr_release_avg = log_transform(curr_release_avg)

curr_release_max = pd.DataFrame({
    'ds': df_popsike[df_popsike['release_id'] == curr_release]['year'],
    'y': df_popsike[df_popsike['release_id'] == curr_release]['max']
})
curr_release_avg = log_transform(curr_release_avg)

In [ ]:
def model(df_in):
    
    m = Prophet()
    m.fit(curr_release_min)
    future = m.make_future_dataframe(periods=8, freq='Y')
    forecast = m.predict(future)
    forecast = inverse_log_transform(forecast)
    forecast = forecast[['ds', 'yhat']].copy()
    mask = (forecast['ds'] > '2019-1-1')
    
    return forecast[mask]


In [ ]:
min_hat = model(curr_release_min)
avg_hat = model(curr_release_avg)
max_hat = model(curr_release_max)

In [ ]:
min_hat.rename(columns={
    'ds': 'year',
    'yhat': 'min'
}, inplace=True)

avg_hat.rename(columns={
    'ds': 'year',
    'yhat': 'avg'
}, inplace=True)

max_hat.rename(columns={
    'ds': 'year',
    'yhat': 'max'
}, inplace=True)

In [ ]:
df_merge = min_hat.merge(avg_hat, on='year')
df_merge = df_merge.merge(max_hat, on='year')

In [ ]:
df_merge

In [ ]:
future = m.make_future_dataframe(periods=8, freq='Y')
forecast = m.predict(future)
forecast = inverse_log_transform(forecast)
forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [ ]:
forecast = forecast[['ds', 'yhat']].copy()

In [ ]:
mask = (forecast['ds'] > '2019-1-1')

In [ ]:
list(forecast[mask]['yhat'])